# Lab 1: 환경 설정

AWS Bedrock 연결, 샘플 데이터 로드, 워크샵 전체에서 사용할 출력 저장 구조를 설정합니다.

## 사전 준비 (터미널에서 실행)

워크샵을 시작하기 전에 터미널에서 아래 명령어를 실행하여 가상환경을 설정하세요.

```bash
# 워크샵 폴더로 이동
cd translation-quality-workshop

# uv로 가상환경 생성
uv venv

# 가상환경 활성화
source .venv/bin/activate

# 필요 패키지 설치
uv pip install ipykernel boto3 pandas matplotlib

# Jupyter 커널 등록
python -m ipykernel install --user --name=workshop --display-name="Workshop"
```

## 설치 완료 후

1. VS Code에서 이 .ipynb 파일 열기
2. 우측 상단 커널 선택 버튼 클릭
3. Python Environments... 선택
4. .venv/bin/python 선택

 ## 학습 목표



 - AWS Bedrock 연결 및 Claude 모델 호출 방법 이해

 - 삼성클라우드 FAQ JSON 데이터 구조 파악

 - 워크샵 전체에서 사용할 출력 저장 구조 설정



 ## 워크샵 전체 흐름



 Lab 1: 환경 설정 (현재)

 ↓

 Lab 2: 번역 기법 비교 (Zero-shot → Few-shot → 용어집)

 ↓

 Lab 3: 품질 평가 - 충실도

 ↓

 Lab 4: 품질 평가 - 용어 일관성

 ↓

 Lab 5: 품질 평가 - 문화/톤

 ↓

 Lab 6: 피드백 기반 재번역

 ↓

 Lab 7: 종합 분석



 ---

 ## 1.1 라이브러리 설치 및 임포트

In [1]:
import json
import os
from datetime import datetime

import boto3
import pandas as pd


 ---

 ## 1.2 AWS Bedrock 클라이언트 초기화



 AWS Bedrock에 연결하여 Claude 모델을 호출할 수 있는 클라이언트를 생성합니다.

In [2]:
# Bedrock 클라이언트 생성
REGION = "us-west-2"

bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=REGION
)

print(f"Bedrock 클라이언트 초기화 완료 (리전: {REGION})")


Bedrock 클라이언트 초기화 완료 (리전: us-west-2)


 ### 사용할 Claude 모델 정의



 이 워크샵에서는 Claude Sonnet 4를 기본 모델로 사용합니다.

 필요에 따라 Haiku(빠름/저렴) 또는 Opus(고품질)로 변경할 수 있습니다.

In [3]:
# 모델 설정
CLAUDE_MODELS = {
    "sonnet": {
        "id": "us.anthropic.claude-sonnet-4-20250514-v1:0",
        "name": "Claude Sonnet 4",
        "description": "균형 잡힌 성능 - 워크샵 기본 모델"
    },
    "haiku": {
        "id": "us.anthropic.claude-haiku-4-20250514-v1:0",
        "name": "Claude Haiku 4",
        "description": "빠른 속도, 낮은 비용 - 대량 처리용"
    }
}

# 기본 모델 선택
DEFAULT_MODEL = "sonnet"
MODEL_ID = CLAUDE_MODELS[DEFAULT_MODEL]["id"]

print(f"기본 모델: {CLAUDE_MODELS[DEFAULT_MODEL]['name']}")
print(f"모델 ID: {MODEL_ID}")


기본 모델: Claude Sonnet 4
모델 ID: us.anthropic.claude-sonnet-4-20250514-v1:0


 ---

 ## 1.3 Claude API 호출 테스트



 Bedrock을 통해 Claude 모델이 정상적으로 호출되는지 간단한 테스트를 수행합니다.

In [4]:
def call_claude(prompt: str, model_id: str = MODEL_ID, max_tokens: int = 1024) -> str:
    """
    Claude 모델을 호출하여 응답을 반환합니다.

    Args:
        prompt: 입력 프롬프트
        model_id: 사용할 모델 ID
        max_tokens: 최대 출력 토큰 수

    Returns:
        모델 응답 텍스트
    """
    request_body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": max_tokens,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ]
    }

    response = bedrock_client.invoke_model(
        modelId=model_id,
        body=json.dumps(request_body),
        contentType="application/json",
        accept="application/json"
    )

    response_body = json.loads(response["body"].read())
    return response_body["content"][0]["text"]


In [5]:
# API 호출 테스트
test_prompt = "안녕하세요를 영어로 번역해주세요. 번역 결과만 출력하세요."
test_response = call_claude(test_prompt)

print("API 호출 테스트")
print(f"입력: {test_prompt}")
print(f"출력: {test_response}")
print("\n✓ Bedrock 연결 성공!")


API 호출 테스트
입력: 안녕하세요를 영어로 번역해주세요. 번역 결과만 출력하세요.
출력: Hello.

✓ Bedrock 연결 성공!


 ---

 ## 1.4 샘플 데이터 로드



 삼성클라우드 FAQ JSON 파일을 로드하고 구조를 확인합니다.

In [6]:
# 데이터 디렉토리 경로
DATA_DIR = "data"

# JSON 파일 로드 함수
def load_json(filename: str) -> dict:
    """JSON 파일을 로드합니다."""
    filepath = os.path.join(DATA_DIR, filename)
    with open(filepath, "r", encoding="utf-8") as f:
        return json.load(f)


In [7]:
# 세 가지 언어 버전 로드
ko_faq = load_json("ko_faq.json")
en_us_faq = load_json("en-rUS_faq.json")
en_gb_faq = load_json("en-rGB_faq.json")

print(f"한국어 FAQ 항목 수: {len(ko_faq)}")
print(f"미국 영어 FAQ 항목 수: {len(en_us_faq)}")
print(f"영국 영어 FAQ 항목 수: {len(en_gb_faq)}")


한국어 FAQ 항목 수: 393
미국 영어 FAQ 항목 수: 393
영국 영어 FAQ 항목 수: 393


 ### 데이터 구조 확인



 JSON 파일은 `key: value` 형태로 구성되어 있습니다.

 - key: FAQ 항목 ID (예: `IDS_FAQ_SC_ABOUT`)

 - value: 해당 언어로 된 텍스트

In [8]:
# 샘플 데이터 확인
sample_keys = list(ko_faq.keys())[:5]

print("샘플 데이터 구조:")
print("-" * 80)

for key in sample_keys:
    print(f"\n[{key}]")
    print(f"  한국어: {ko_faq.get(key, 'N/A')[:80]}...")
    print(f"  영어(US): {en_us_faq.get(key, 'N/A')[:80]}...")


샘플 데이터 구조:
--------------------------------------------------------------------------------

[IDS_FAQ_SC_ABOUT]
  한국어: 삼성 클라우드는 사용자의 삼성 계정과 연동된 정보에 대한 동기화와 사용자 단말에 저장된 정보의 백업/복원을 지원하는 서비스입니다....
  영어(US): Samsung Cloud is a service that supports the synchronization of information link...

[IDS_FAQ_SC_SUPPORTED_HEADER_01]
  한국어: 동기화/백업 지원 항목은 다음과 같습니다....
  영어(US): The supported items for sync/backup are as follows....

[IDS_FAQ_SC_SUPPORTED_TEXT_OL_01]
  한국어: 동기화 지원 항목 : 삼성 계정으로 저장된 일정(캘린더) 및 연락처, 삼성 인터넷, Samsung Notes, Samsung Pass, 갤러리,...
  영어(US): Items supported by sync: Events (Calendar) and Contacts saved in your Samsung ac...

[IDS_FAQ_SC_SUPPORTED_TEXT_OL_02]
  한국어: 백업/복원 지원 항목 : 단말에 저장된 일정(캘린더) 및 연락처, 통화 기록, 시계, 메시지, 홈 화면, 애플리케이션, 단말의 설정, 음성 녹음...
  영어(US): Items supported by backup/restore: Events (Calendar) and Contacts saved on your ...

[IDS_FAQ_SC_SUPPORTED_HEADER_02]
  한국어: 위 서비스들이 지원되는 단말은 다음과 같습니다....
  영어(US): The services outlined above are supported by the followin

 ### DataFrame으로 변환



 분석 및 비교를 위해 pandas DataFrame으로 변환합니다.

In [9]:
# DataFrame 생성
def create_faq_dataframe(ko_data: dict, en_us_data: dict, en_gb_data: dict) -> pd.DataFrame:
    """세 가지 언어 버전을 하나의 DataFrame으로 통합합니다."""
    rows = []

    for key in ko_data.keys():
        rows.append({
            "id": key,
            "ko": ko_data.get(key, ""),
            "en_us": en_us_data.get(key, ""),
            "en_gb": en_gb_data.get(key, "")
        })

    return pd.DataFrame(rows)

faq_df = create_faq_dataframe(ko_faq, en_us_faq, en_gb_faq)
print(f"전체 FAQ 항목 수: {len(faq_df)}")
faq_df.head()


전체 FAQ 항목 수: 393


,id,ko,en_us,en_gb
0,IDS_FAQ_SC_ABOUT,삼성 클라우드는 사용자의 삼성 계정과 연동된 정보에 대한 동기화와 사용자 단말에 저...,Samsung Cloud is a service that supports the s...,Samsung Cloud is a service that supports the s...
1,IDS_FAQ_SC_SUPPORTED_HEADER_01,동기화/백업 지원 항목은 다음과 같습니다.,The supported items for sync/backup are as fol...,The supported items for sync/backup are as fol...
2,IDS_FAQ_SC_SUPPORTED_TEXT_OL_01,"동기화 지원 항목 : 삼성 계정으로 저장된 일정(캘린더) 및 연락처, 삼성 인터넷,...",Items supported by sync: Events (Calendar) and...,Items supported by sync: Events (Calendar) and...
3,IDS_FAQ_SC_SUPPORTED_TEXT_OL_02,"백업/복원 지원 항목 : 단말에 저장된 일정(캘린더) 및 연락처, 통화 기록, 시계...",Items supported by backup/restore: Events (Cal...,Items supported by backup/restore: Events (Cal...
4,IDS_FAQ_SC_SUPPORTED_HEADER_02,위 서비스들이 지원되는 단말은 다음과 같습니다.,The services outlined above are supported by t...,The services outlined above are supported by t...


 ### US vs GB 영어 차이 확인



 동일한 원문에 대해 미국 영어와 영국 영어 번역이 어떻게 다른지 확인합니다.

In [10]:
# US와 GB가 다른 항목 찾기
diff_items = faq_df[faq_df["en_us"] != faq_df["en_gb"]]
print(f"US/GB 번역이 다른 항목 수: {len(diff_items)}")

if len(diff_items) > 0:
    print("\n차이가 있는 항목 예시:")
    print("-" * 80)
    sample = diff_items.iloc[0]
    print(f"ID: {sample['id']}")
    print(f"한국어: {sample['ko']}")
    print(f"영어(US): {sample['en_us']}")
    print(f"영어(GB): {sample['en_gb']}")


US/GB 번역이 다른 항목 수: 76

차이가 있는 항목 예시:
--------------------------------------------------------------------------------
ID: IDS_FAQ_SC_MAIN_HEADER_02
한국어: 삼성 클라우드를 사용하려면 삼성 계정이 필요한가요?
영어(US): Do I need a Samsung account to use Samsung Cloud?
영어(GB): Do I need my Samsung account to use Samsung Cloud?


 ---

 ## 1.5 워크샵에서 사용할 FAQ 항목 선정



 전체 FAQ 중 워크샵 실습에 사용할 항목을 선정합니다.

 다양한 유형의 텍스트를 포함하도록 선정합니다.

In [20]:
# 워크샵 실습용 FAQ 항목 선정
WORKSHOP_FAQ_IDS = [
    # 1. 톤 차이 (a vs my)
    "IDS_FAQ_SC_MAIN_HEADER_02",
    
    # 2. 동사 선택 차이 (remove vs delete)
    "IDS_FAQ_SC_MAIN_HEADER_03",
    
    # 3. 철자 차이 (inquiry vs enquiry) + 전치사 차이
    "IDS_FAQ_SC_ACCESSING_TEXT",
    
    # 4. 철자 차이 (synchronized vs synchronised)
    "IDS_FAQ_SC_MAIN_HEADER_13",
    
    # 5. 용어 차이 (Trash vs Recycle bin)
    "IDS_FAQ_GO_MAIN_HEADER_27",
    
    # 6. 복합 차이 (carriers vs networks, 문장 구조)
    "IDS_FAQ_GO_DEVICE_NOTES_UL_01",
]


# 선정된 항목 추출
workshop_df = faq_df[faq_df["id"].isin(WORKSHOP_FAQ_IDS)].copy()
workshop_df = workshop_df.reset_index(drop=True)

print(f"워크샵 실습용 FAQ 항목: {len(workshop_df)}개")
print("-" * 80)

for idx, row in workshop_df.iterrows():
    print(f"\n[{idx + 1}] {row['id']}")
    print(f"    한국어: {row['ko'][:60]}...")


워크샵 실습용 FAQ 항목: 6개
--------------------------------------------------------------------------------

[1] IDS_FAQ_SC_MAIN_HEADER_02
    한국어: 삼성 클라우드를 사용하려면 삼성 계정이 필요한가요?...

[2] IDS_FAQ_SC_MAIN_HEADER_03
    한국어: 삼성 계정을 단말에서 제거하면 어떻게 되나요?...

[3] IDS_FAQ_SC_ACCESSING_TEXT
    한국어: 삼성 클라우드 웹페이지 접속 후 Contact us를 통해 '1대1 문의하기'로 문의해주시면 사용자 정보 확...

[4] IDS_FAQ_SC_MAIN_HEADER_13
    한국어: 갤럭시 단말이 현재 없습니다. 삼성 서비스에 저장 및 동기화했던 데이터나 정보를 확인할 수 있는 방법이 있나...

[5] IDS_FAQ_GO_DEVICE_NOTES_UL_01
    한국어: 특정 단말, 국가, 사업자에 따라 지원 여부는 다를 수 있습니다....

[6] IDS_FAQ_GO_MAIN_HEADER_27
    한국어: 어디로 가면 클라우드 휴지통을 확인할 수 있나요?...


 ---

 ## 1.6 출력 저장 구조 설정



 워크샵 전체에서 사용할 결과 저장 구조를 설정합니다.

 각 Lab의 결과는 `lab_outputs/` 디렉토리에 JSON 파일로 저장됩니다.

In [12]:
# 출력 디렉토리 생성
OUTPUT_DIR = "lab_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"출력 디렉토리 생성: {OUTPUT_DIR}/")


출력 디렉토리 생성: lab_outputs/


 ### 결과 저장/로드 함수 정의



 모든 Lab에서 공통으로 사용할 저장/로드 함수를 정의합니다.

In [13]:
def save_results(data: dict, filename: str) -> str:
    """
    결과를 JSON 파일로 저장합니다.

    Args:
        data: 저장할 데이터 (dict)
        filename: 파일명 (예: "translations_zero_shot.json")

    Returns:
        저장된 파일 경로
    """
    filepath = os.path.join(OUTPUT_DIR, filename)

    # 메타데이터 추가
    data["_metadata"] = {
        "saved_at": datetime.now().isoformat(),
        "model": CLAUDE_MODELS[DEFAULT_MODEL]["name"]
    }

    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"✓ 저장 완료: {filepath}")
    return filepath

def load_results(filename: str) -> dict:
    """
    저장된 JSON 파일을 로드합니다.

    Args:
        filename: 파일명 (예: "translations_zero_shot.json")

    Returns:
        로드된 데이터 (dict)
    """
    filepath = os.path.join(OUTPUT_DIR, filename)

    if not os.path.exists(filepath):
        raise FileNotFoundError(f"파일을 찾을 수 없습니다: {filepath}")

    with open(filepath, "r", encoding="utf-8") as f:
        data = json.load(f)

    print(f"✓ 로드 완료: {filepath}")
    return data


 ### 결과 스키마 정의



 각 Lab에서 저장할 결과의 스키마를 정의합니다.

In [14]:
# 번역 결과 스키마
TRANSLATION_SCHEMA = {
    "id": "FAQ 항목 ID",
    "source": "원문",
    "source_lang": "원문 언어 (ko/en)",
    "target_lang": "목표 언어 (ko/en)",
    "translation": "번역 결과",
    "method": "번역 기법 (zero_shot/few_shot/glossary)",
    "timestamp": "번역 시각"
}

# 평가 결과 스키마
EVALUATION_SCHEMA = {
    "id": "FAQ 항목 ID",
    "source": "원문",
    "translation": "번역문",
    "score": "평가 점수 (0-5)",
    "verdict": "판정 (Pass/Fail/Review)",
    "issues": "발견된 문제점 목록",
    "suggestion": "수정 제안",
    "reasoning": "평가 근거",
    "eval_type": "평가 유형 (faithfulness/terminology/culture_tone)",
    "timestamp": "평가 시각"
}

print("번역 결과 스키마:")
for key, desc in TRANSLATION_SCHEMA.items():
    print(f"  - {key}: {desc}")

print("\n평가 결과 스키마:")
for key, desc in EVALUATION_SCHEMA.items():
    print(f"  - {key}: {desc}")


번역 결과 스키마:
  - id: FAQ 항목 ID
  - source: 원문
  - source_lang: 원문 언어 (ko/en)
  - target_lang: 목표 언어 (ko/en)
  - translation: 번역 결과
  - method: 번역 기법 (zero_shot/few_shot/glossary)
  - timestamp: 번역 시각

평가 결과 스키마:
  - id: FAQ 항목 ID
  - source: 원문
  - translation: 번역문
  - score: 평가 점수 (0-5)
  - verdict: 판정 (Pass/Fail/Review)
  - issues: 발견된 문제점 목록
  - suggestion: 수정 제안
  - reasoning: 평가 근거
  - eval_type: 평가 유형 (faithfulness/terminology/culture_tone)
  - timestamp: 평가 시각


 ### 저장 테스트

In [21]:
# 저장 테스트
test_data = {
    "test": "워크샵 저장 테스트",
    "items": ["item1", "item2"]
}

save_results(test_data, "test_save.json")

# 로드 테스트
loaded_data = load_results("test_save.json")
print(f"로드된 데이터: {loaded_data['test']}")

# 테스트 파일 삭제
os.remove(os.path.join(OUTPUT_DIR, "test_save.json"))
print("✓ 테스트 파일 삭제 완료")


✓ 저장 완료: lab_outputs/test_save.json
✓ 로드 완료: lab_outputs/test_save.json
로드된 데이터: 워크샵 저장 테스트
✓ 테스트 파일 삭제 완료


 ---

 ## 1.7 워크샵 데이터 저장



 다음 Lab에서 사용할 워크샵 데이터를 저장합니다.

In [22]:
# 워크샵 기본 데이터 저장
workshop_data = {
    "faq_items": workshop_df.to_dict(orient="records"),
    "config": {
        "default_model": DEFAULT_MODEL,
        "model_id": MODEL_ID,
        "region": REGION,
        "source_lang": "ko",
        "target_lang": "en"
    }
}

save_results(workshop_data, "workshop_config.json")


✓ 저장 완료: lab_outputs/workshop_config.json


'lab_outputs/workshop_config.json'

 ---

 ## 1.8 환경 설정 요약



 Lab 1에서 설정한 내용을 요약합니다.

In [23]:
print("=" * 80)
print("Lab 1: 환경 설정 완료")
print("=" * 80)

print("\n[AWS Bedrock 설정]")
print(f"  - 리전: {REGION}")
print(f"  - 기본 모델: {CLAUDE_MODELS[DEFAULT_MODEL]['name']}")
print(f"  - 모델 ID: {MODEL_ID}")

print("\n[데이터 설정]")
print(f"  - 전체 FAQ 항목: {len(faq_df)}개")
print(f"  - 워크샵 실습 항목: {len(workshop_df)}개")
print(f"  - 지원 언어: 한국어, 영어(US), 영어(GB)")

print("\n[출력 설정]")
print(f"  - 출력 디렉토리: {OUTPUT_DIR}/")
print(f"  - 저장 파일: workshop_config.json")

print("\n[다음 단계]")
print("  → Lab 2: 번역 기법 비교 (Zero-shot, Few-shot, 용어집)")

print("\n" + "=" * 80)


Lab 1: 환경 설정 완료

[AWS Bedrock 설정]
  - 리전: us-west-2
  - 기본 모델: Claude Sonnet 4
  - 모델 ID: us.anthropic.claude-sonnet-4-20250514-v1:0

[데이터 설정]
  - 전체 FAQ 항목: 393개
  - 워크샵 실습 항목: 6개
  - 지원 언어: 한국어, 영어(US), 영어(GB)

[출력 설정]
  - 출력 디렉토리: lab_outputs/
  - 저장 파일: workshop_config.json

[다음 단계]
  → Lab 2: 번역 기법 비교 (Zero-shot, Few-shot, 용어집)

